
# Реализация Шифрования blowfish

# c CTR

In [1]:
import math
from mpmath import mp
import more_itertools
import time
import logging
import os
import sys
import copy
import datetime
import random
import secrets

In [2]:
def resource_path(relative):
    '''
    :return полный путь до файла
    '''
    if hasattr(sys, '_MEIPASS'):
        return os.path.join(sys._MEIPASS, relative)
    else:
        return os.path.join(os.path.abspath("."), relative)

In [3]:
add_suf = 0

In [4]:
#Выполнить один раз
os.chdir('Include')  

In [5]:
# os.chdir('Include') 
PATH_KEY = "KEY.txt"
PATH_INPUT_FILE = "MESSAGE.txt"
PATH_DECRYPT_FILE = "DECRYPT.txt"
PATH_INCRYPT_FILE = "INCRYPT.txt"

In [6]:
LEN_PI = 20000 #Длина Числа ПИ
mp.dps = LEN_PI #Установка числа длины ПИ
PI = int((mp.pi - int(mp.pi))*10**LEN_PI) #Берем мантису числа ПИ
PI_HEX = hex(PI)[2:8336+2] #Переводим в 16 систему

In [7]:
FIXED_P = [''.join(i) for i in more_itertools.grouper(PI_HEX[:8*18],8)] #Матрица Раундовых ключей

In [8]:
#Матрица подстановок
FIXED_S = [i for i in more_itertools.grouper([''.join(i) for i in more_itertools.grouper(PI_HEX[8*18:],8)],256)] 
FIXED_S = [list(i) for i in FIXED_S]

In [9]:
def F(block32:str, S:list):
    '''
    Функция итерации
    :param block32: Входной блок 32бит 
    :paran S: Матрица подстановок
    :return: Результат функции итерации
    '''
    block32 = '{0:08x}'.format(int(block32,16))

    X1 = int(block32[0:2],16)
    X2 = int(block32[2:4],16)
    X3 = int(block32[4:6],16)
    X4 = int(block32[6:8],16)
    
    return hex(((((int(S[0][X1],16) + int(S[1][X2],16) ) % 2**32 ) ^ int(S[2][X3],16)) + int(S[3][X4],16)) % 2**32)[2:]

In [10]:
def blowfishEncryptBlock(left32:str, right32:str, P:list, S:list)->list:
    '''
    :param left32: Левые 32 бита входного 64 блока
    :param right32: Правые 32 бита входного 64 блока
    :param P: Раундовые ключи
    :param S: Подстановочные ключи
    :return: Зашифрованный 64 бит блок
    '''
    for i in range(16):
        left32 = '{0:08x}'.format(int(left32,16) ^ int(P[i],16))
        right32 = '{0:08x}'.format(int(F(str(left32), S),16) ^ int(right32,16))
        left32, right32 = right32, left32
        
    left32, right32 = right32, left32
    right32 = (int(right32,16) ^ int(P[16], 16))
    left32 = (int(left32,16) ^ int(P[17], 16))
    
    return '{0:08x}'.format(left32),'{0:08x}'.format(right32)

In [11]:
def blowfishDecryptBlock(left32:str, right32:str, P:list, S:list)->list:
    '''
    :param left32: Левые 32 бита входного 64 блока
    :param right32: Правые 32 бита входного 64 блока
    :param P: Раундовые ключи
    :param S: Подстановочные коючи
    :return: Расшифрованный 64 бит блок
    '''  
    
    for i in range(17,1,-1):
        left32 = '{0:08x}'.format(int(left32,16) ^ int(P[i],16))
        right32 = '{0:08x}'.format(int(F(str(left32), S),16) ^ int(right32,16))
        left32, right32 = right32, left32
        
    left32, right32 = right32, left32
    right32 = '{0:08x}'.format(int(right32,16) ^ int(P[1], 16))
    left32 = '{0:08x}'.format(int(left32,16) ^ int(P[0], 16))
    
    return left32, right32

In [12]:
def generateKeyBytes(key:str, len_result_key:int)->list:
    '''
    Функция переводи ключ в список байт. Если ключ меньше длины раундовых ключей, конкатинируем -> keykeykey
    :param key:  Входной ключ
    :param len_result_key: Длина выходного ключа, кол-во блоков
    : Результат создания ключа
    '''
    result = []
    key = key if key else 0
    for i in range(len_result_key * 4): 
        result.append(
            bytes(key[i%len(key)],encoding='cp1251').hex()
        )
    
    return [''.join(i) for i in more_itertools.grouper(result,4)]

In [13]:
def generateTextBytes(text:str, len_block:int = 8)->list:
    '''
    Функция переводит текст в список байт. Если текст меньше длины кратной len_block то дополняем '0'
    :param text:  Входной текст
    :param len_block: Длина блока в битах
    : Результат создания списка текста 64бит
    '''
    global add_suf
    add_suf = -len(text)%len_block
#     print(add_suf)
    result = []
    text = text +'0'*(add_suf)
#     print(len(text))
    for i in range(len(text)): 
        result.append(
            bytes(text[i],encoding='cp1251').hex()
        )
#     print(result)
    return [''.join(i) for i in more_itertools.grouper(result,8)]

In [14]:
def generateRoundKey(P:list, key_bytes:list):
    '''
    Функция создающая раундовые ключи. Берет раундовый ключи P1 P2 ... и XOR c ключом
    :param P: Раундовые ключи
    :param key: Ключ
    : Результат создания ключа
    '''
    result_p = []
    for p,k in zip(P, key_bytes):
        result_p.append(
            hex((int(p,16)) ^ (int(k,16)))[2:]
        )
    
    return result_p

In [15]:
def generateAllKey(P:list, S:list):
    '''
    Функция создающая итоговые Раундовые и Подстановочные ключи
    :param P: Раундовые ключи
    :param S: Подстановочные ключи
    : Результат - Раундовые и Подстановочные ключи
    '''
    NEW_P = copy.deepcopy(P)
    NEW_S = copy.deepcopy(S)
    
    tmp = None
    for i in range(0,17,2):
        if not tmp:
                tmp = blowfishEncryptBlock(left32='0000', right32='0000', P=P, S=S)
        else:
            tmp = blowfishEncryptBlock(left32=tmp[0], right32=tmp[1], P=P, S=S)
            
        P[i] = tmp[0]
        P[i+1] = tmp[1]
        
    for i in range(4):
        for j in range(0,255,2): 
            tmp = blowfishEncryptBlock(left32=tmp[0], right32=tmp[1], P=P, S=S)
            S[i][j] = tmp[0] 
            S[i][j+1] = tmp[1]
            
    return P, S

In [16]:
def generateIV() -> list:
    '''
    :return: Возвращает уникальную гамму размером 32 бит
    '''
    return secrets.token_hex(4) + '0'*8

In [17]:
def encrypt(text_bytes:list,P:list,S:list, IV:str = None):
    '''
    Шифрование текста
    :param text_bytes: Открытый текст
    :param P:Матрица раундовых ключей
    :param S:Матрица подстановок
    :param IV: Блок гаммы длинной 64 бит
    :return: Закрытый текс
    '''
    if not IV:
        encrypt_text = ''
        for i in text_bytes:
            tmp = blowfishEncryptBlock(left32=i[0:8],right32=i[8:],P=P,S=S)
            encrypt_text+=(tmp[0]+tmp[1])
    else:
        CTR = IV
        encrypt_text = ''
        for i in text_bytes:
            IVL,IVR  = blowfishEncryptBlock(left32=CTR[0:8],right32=CTR[8:],P=P,S=S)
            encrypt_text += '{0:016x}'.format(int(IVL+IVR,16) ^ int(i,16))
            CTR = '{0:016x}'.format(int(CTR,16)+1)
            
    return encrypt_text

In [18]:
def decrypt(text_bytes:list,P:list,S:list, IV:list = None):
    '''
    Шифрование текста
    :param text_bytes: Открытый текст
    :param P:Матрица раундовых ключей
    :param S:Матрица подстановок
    :param IV: Блок гаммы
    :return: Открытый текс
    '''
    if not IV:
        decrypt_text = ''
        for i in text_bytes:
            tmp = blowfishDecryptBlock(left32=i[0:8],right32=i[8:],P=P,S=S)
            decrypt_text+=(tmp[0]+tmp[1])
    else:
        CTR = IV
        decrypt_text = ''
        for i in text_bytes:
            IVL,IVR  = blowfishDecryptBlock(left32=CTR[0:8],right32=CTR[8:],P=P,S=S)
            decrypt_text += '{0:016x}'.format(int(IVL+IVR,16) ^ int(i,16))
            CTR = '{0:016x}'.format(int(CTR,16)+1)
            
    return decrypt_text

In [19]:
#Считываем ключ, переводим в список 32 бит, генерируем раундовые ключи
with open(PATH_KEY,'r',encoding='cp1251') as file:
    key = file.read()
    key_bytes = generateKeyBytes(key, len_result_key=len(FIXED_P))
    NEW_FIXED_P = generateRoundKey(P=FIXED_P, key_bytes=key_bytes)
NEW_FIXED_P

['1ac7bc5a',
 '6d4a2ec0',
 '13c4c18d',
 'a3474245',
 '847fb617',
 '79a2bb1f',
 'dd785b47',
 'f660996e',
 'c07cb106',
 'b9de7ca6',
 '3bc85a50',
 '84532a69',
 'b6374581',
 '37d7cafe',
 'e2a71779',
 '6ce43103',
 'ba537681',
 '6a25cd67']

In [20]:
#Считываваем по 64 бит Входного текста
with open(PATH_INPUT_FILE,'r',encoding='cp1251') as file:
    text = file.read()
    text_bytes = generateTextBytes(text)
text_bytes

['d1eecee1f9e5ede8',
 'e546647367647367',
 'cfeef2e0e9ede8ea',
 '3132336464646464',
 '6464643030303030']

In [21]:
#Генерируем раундовые и подстановочные ключи
NEW_FIXED_P, NEW_FIXED_S = generateAllKey(NEW_FIXED_P,FIXED_S)
NEW_FIXED_P, NEW_FIXED_S

(['027b29e7',
  'be392a73',
  '518c2d7f',
  'e70d5399',
  '9944a345',
  '26849d98',
  'f4095bf4',
  '8fc77367',
  '63f14a01',
  '053bb284',
  'd79710be',
  '52f777bf',
  'fae5d358',
  '2280f1d9',
  '5b23dcaf',
  'a2f13b60',
  '9ab5eedb',
  '8d78512d'],
 [['ff0cca2d',
   'e2b35730',
   'f8c13c69',
   '5246da47',
   '0ebe26d8',
   'b9a76ccc',
   '82dccefe',
   'af167b89',
   '408f75a8',
   '2aa470c6',
   'c3a118a6',
   '03406e80',
   'f7f344e4',
   '3b31fb9a',
   '21a8b759',
   '1b757894',
   'd7b475f9',
   '984e7361',
   '02ac9762',
   'df7be5ed',
   'd6829a55',
   'ba84cb6f',
   '20a5fb99',
   'c84bc237',
   '1fd6bfc9',
   'e0ac27a7',
   'b787a869',
   '883d8e60',
   'ff2f8c0d',
   'bf6850c2',
   'a0a43c88',
   '929edd6a',
   '5dbf8598',
   'd78ab9f5',
   '7c9dbb7a',
   'bb1fc97b',
   '2525fdbb',
   '85cf0a17',
   '9567aa1f',
   '6bf631e6',
   '2f8a96aa',
   '19767313',
   'a4651744',
   '19677939',
   '5adfd09a',
   'be6fa3a5',
   '4700768b',
   '09ff10e2',
   'd77af1e1',
   '501510a3

In [22]:
#делаем шифрацию, вначало записываем add_suf - отвечает за кол-во дополненых 0 до кратности 64 битам
with open(PATH_INCRYPT_FILE,'w',encoding='cp1251') as file:
    global add_suf
    IV = generateIV()
    encrypt_text = encrypt(text_bytes=text_bytes,P=NEW_FIXED_P,S=NEW_FIXED_S, IV=IV)
    file.write(str(add_suf))
    file.write(encrypt_text)
    file.write('\n')
    file.write(''.join(IV))
encrypt_text

'e7c3ae2c408a0d47e4694116c513fe0b89a5bde246a0c692afcd435086fe9770eac824bcc9a804d6'

In [23]:
#Считываем файл где лежит зашифрованный текст
with open(PATH_INCRYPT_FILE,'r',encoding='cp1251') as file:
    encrypt_text = file.readlines()
    add_suf = int(encrypt_text[0][0])
    IV = [''.join(i) for i in more_itertools.grouper(encrypt_text[1],16)][0]
    encrypt_text = encrypt_text[0][1:-1]

In [24]:
encrypt_text, IV

('e7c3ae2c408a0d47e4694116c513fe0b89a5bde246a0c692afcd435086fe9770eac824bcc9a804d6',
 '0dbcb1e800000000')

In [25]:
#Записываем результат дешифрации
with open(PATH_DECRYPT_FILE,'w',encoding='cp1251') as file:
    decrypt_text = (encrypt(text_bytes=[''.join(i) for i in more_itertools.grouper(encrypt_text,16)],P=NEW_FIXED_P,S=NEW_FIXED_S, IV=IV))
    for i in range(0,len(decrypt_text)-int(add_suf)*2,2):
        file.write(
            bytes.fromhex(decrypt_text[i]+decrypt_text[i+1]).decode('cp1251')
        )
decrypt_text

'd1eecee1f9e5ede8e546647367647367cfeef2e0e9ede8ea31323364646464646464643030303030'

In [26]:
print('Открытый текст = ',text_bytes, '\nРасшифрованный текст = ', [''.join(i) for i in more_itertools.grouper(decrypt_text,16)])

Открытый текст =  ['d1eecee1f9e5ede8', 'e546647367647367', 'cfeef2e0e9ede8ea', '3132336464646464', '6464643030303030'] 
Расшифрованный текст =  ['d1eecee1f9e5ede8', 'e546647367647367', 'cfeef2e0e9ede8ea', '3132336464646464', '6464643030303030']


# Тест по времени

In [27]:
import random

### 1 блок

In [29]:
text_bytes = ['1abc'*4]
start = datetime.datetime.now()
IV = generateIV()
encrypt_text = encrypt(text_bytes=text_bytes,P=NEW_FIXED_P,S=NEW_FIXED_S,IV=IV)
finish_encrypt = datetime.datetime.now()
decrypt_text = decrypt(text_bytes=[''.join(i) for i in more_itertools.grouper(encrypt_text,16)],P=NEW_FIXED_P,S=NEW_FIXED_S,IV=IV)
finish = datetime.datetime.now()
print(f'''
        Open_text = {text_bytes[0]}
        Encrypt_text = {encrypt_text}
        Decrypt_text = {decrypt_text}
        time_start = {start}
        time_finish = {finish}
        ALL_lambda = {finish -start}
        time_encrypt = {finish_encrypt - start}
        time_decrypt = {finish - finish_encrypt}
        ''')


        Open_text = 1abc1abc1abc1abc
        Encrypt_text = dc9fbd039b6c77b7
        Decrypt_text = 0b45b22081b8d998
        time_start = 2020-03-27 16:21:52.207061
        time_finish = 2020-03-27 16:21:52.207061
        ALL_lambda = 0:00:00
        time_encrypt = 0:00:00
        time_decrypt = 0:00:00
        


### 1000блоков

In [30]:
text_bytes = ['1abc'*4]*1000
IV = generateIV()
start = datetime.datetime.now()
encrypt_text = encrypt(text_bytes=text_bytes,P=NEW_FIXED_P,S=NEW_FIXED_S,IV=IV)
finish_encrypt = datetime.datetime.now()
decrypt_text = decrypt(text_bytes=[''.join(i) for i in more_itertools.grouper(encrypt_text,16)],P=NEW_FIXED_P,S=NEW_FIXED_S,IV=IV)
finish = datetime.datetime.now()
print(f'''
        time_start = {start}
        time_finish = {finish}
        ALL_lambda = {finish -start}
        time_encrypt = {finish_encrypt - start}
        time_decrypt = {finish - finish_encrypt}
        ''')


        time_start = 2020-03-27 16:21:55.871467
        time_finish = 2020-03-27 16:21:56.202347
        ALL_lambda = 0:00:00.330880
        time_encrypt = 0:00:00.183487
        time_decrypt = 0:00:00.147393
        


### 100000

In [31]:
text_bytes = ['1abc'*4]*100000
IV = generateIV()
start = datetime.datetime.now()
encrypt_text = encrypt(text_bytes=text_bytes,P=NEW_FIXED_P,S=NEW_FIXED_S, IV=IV)
finish_encrypt = datetime.datetime.now()
decrypt_text = decrypt(text_bytes=[''.join(i) for i in more_itertools.grouper(encrypt_text,16)],P=NEW_FIXED_P,S=NEW_FIXED_S, IV=IV)
finish = datetime.datetime.now()
print(f'''
        time_start = {start}
        time_finish = {finish}
        ALL_lambda = {finish -start}
        time_encrypt = {finish_encrypt - start}
        time_decrypt = {finish - finish_encrypt}
        ''')


        time_start = 2020-03-27 16:22:01.403502
        time_finish = 2020-03-27 16:22:34.908387
        ALL_lambda = 0:00:33.504885
        time_encrypt = 0:00:15.775542
        time_decrypt = 0:00:17.729343
        
